### 대화 메모리 관리
- 대화 메모리(MessageHistory)는 AI Agent가 이전 대화 내용을 메시지 단위로 저장하고,
이를 바탕으로 **맥락(Context)**을 유지하여 자연스러운 대화를 이어가기 위한 구조이다.
- 단순한 “대화 이력 저장”을 넘어서, 대화 요약(Summarization)과 의도 추적(Context Tracking) 기능을 포함한다.

In [1]:
import os
from dotenv import load_dotenv

# .env 파일의 내용 불러오기
load_dotenv("C:/env/.env")

True

### [1] ConversationBufferMemory: 구버전  
 (LangChain ≥ 0.2.7 신버전에서 Deprecated, 현재 버전에서 실행 오류)

In [5]:
# from langchain_openai import ChatOpenAI
# from langchain.chains import ConversationChain
# from langchain.memory import ConversationBufferMemory

# # 1) LLM 모델 정의
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# # 2) 메모리 객체 생성
# # return_messages=True → 각 대화를 메시지 객체로 반환
# memory = ConversationBufferMemory(return_messages=True)

# # 3) 대화 체인(ConversationChain) 구성
# conversation = ConversationChain(
#     llm=llm,
#     memory=memory,
#     verbose=True   # 내부 동작 로그 출력
# )

# # 4) 대화 실행
# print("=== 대화 시작 ===")
# response1 = conversation.predict(input="안녕? 나는 홍길동이야.")
# print("🤖:", response1)

# response2 = conversation.predict(input="내 이름이 뭐지?")
# print("🤖:", response2)

# response3 = conversation.predict(input="좋아하는 음식은 뭐야?")
# print("🤖:", response3)

## 5) 메모리 내용 확인
# print("\n=== 현재 메모리 저장 내용 ===")
# print(memory.buffer)

# C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7420\1386141394.py:13: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
#   memory = ConversationBufferMemory(return_messages=True)
# C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_7420\1386141394.py:16: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
#   conversation = ConversationChain(

### [2] RunnableWithMessageHistory : 최신 버전 스타일
LangChain 최신 대화 관리 클래스 (≥ 0.2.7) <br>
LLM(예: ChatOpenAI)과 대화 기록(MessageHistory)을 연결해 멀티턴 대화(대화 맥락 유지)를 지원한다.

In [6]:
#  LangChain 최신 메모리 구조 예제
# ConversationChain → RunnableWithMessageHistory로 변경됨

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 1) LLM 모델 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 2) 세션별 대화 기록 저장소
store = {}

def get_session_history(session_id: str):
    """세션 ID별로 대화 기록을 저장하고 불러오는 함수"""
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 3) RunnableWithMessageHistory 구성
conversation = RunnableWithMessageHistory(
    llm,
    get_session_history
)

# 4) 세션 ID 설정
config = {"configurable": {"session_id": "chat-001"}}

# 5) 대화 실행
print("=== 대화 시작 ===")
response1 = conversation.invoke([HumanMessage(content="안녕? 나는 홍길동이야!")], config=config)
print("🤖:", response1.content)

response2 = conversation.invoke([HumanMessage(content="내 이름이 뭐지?")], config=config)
print("🤖:", response2.content)

response3 = conversation.invoke([HumanMessage(content="좋아하는 음식은 뭐야?")], config=config)
print("🤖:", response3.content)

# 6) 현재 메모리(대화 히스토리) 확인
history = get_session_history("chat-001")
print("\n=== 현재 대화 히스토리 ===")
for msg in history.messages:
    role = "🧑 사용자" if msg.type == "human" else "🤖 AI"
    print(f"{role}: {msg.content}")

=== 대화 시작 ===
🤖: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🤖: 당신의 이름은 홍길동입니다! 다른 질문이나 궁금한 점이 있으면 말씀해 주세요.
🤖: 저는 인공지능이라 음식을 먹거나 좋아할 수는 없지만, 많은 사람들이 좋아하는 음식에 대해 이야기할 수 있어요! 예를 들어, 피자, 초밥, 김치찌개 같은 음식들이 인기가 많죠. 홍길동님은 어떤 음식을 좋아하시나요?

=== 현재 대화 히스토리 ===
🧑 사용자: 안녕? 나는 홍길동이야!
🤖 AI: 안녕하세요, 홍길동님! 만나서 반갑습니다. 어떻게 도와드릴까요?
🧑 사용자: 내 이름이 뭐지?
🤖 AI: 당신의 이름은 홍길동입니다! 다른 질문이나 궁금한 점이 있으면 말씀해 주세요.
🧑 사용자: 좋아하는 음식은 뭐야?
🤖 AI: 저는 인공지능이라 음식을 먹거나 좋아할 수는 없지만, 많은 사람들이 좋아하는 음식에 대해 이야기할 수 있어요! 예를 들어, 피자, 초밥, 김치찌개 같은 음식들이 인기가 많죠. 홍길동님은 어떤 음식을 좋아하시나요?


### [3] InMemoryChatMessageHistory
- 대화 기록(Chat Messages)을 메모리(RAM)에 저장하는 기본 클래스
- 한 세션(session_id) 동안 사용자 메시지와 AI 응답을 순차적으로 리스트 형태로 보관
- 프로그램이 종료되면 기록이 사라지는 휘발성(Volatile) 구조
- RunnableWithMessageHistory와 함께 사용해 멀티턴 대화 관리에 활용됨

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1) 메모리 객체 생성
history = InMemoryChatMessageHistory()

